# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)
    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
    

186


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

167


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster


# To establish connection and begin executing queries, need a session
try:
    cluster = Cluster(["127.0.0.1"])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [50]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_library 
    (artist text, 
    itemInSession int, 
    lenght float , 
    level text , 
    sessionId int , 
    song text , 
    PRIMARY KEY (sessionId, itemInSession))
    """)
except Exception as e:
    print("Error creating table")
    print(e)

In [51]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_library (artist, itemInSession, lenght, level, sessionId, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], int(line[3]), float(line[5]), line[6], int(line[8]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [63]:
def select(query):
    """Execute a query to select specific table values"""
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

    for row in rows:
        print (list(row))

In [53]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
select(("SELECT artist, song, lenght FROM song_library WHERE sessionId=338 AND itemInSession=4"))

In [88]:
# Using pandas to validate answer
import pandas as pd
event  = pd.read_csv('event_datafile_new.csv')
event[event['sessionId'] == 138]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

In [60]:
try:
    session.execute("""
   CREATE TABLE IF NOT EXISTS song_library_by_user(
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY (userId, sessionId, itemInSession)
    )
    """)
except Exception as e:
    print("Error creating table")
    print(e)

In [61]:
file = 'event_datafile_new.csv'

# Insert expected data into song_plays_by_user_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library_by_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [65]:
select("SELECT sessionId, artist, firstName, lastName FROM song_library_by_user WHERE userId=10 AND sessionId=182")

[182, 'Down To The Bone', 'Sylvie', 'Cruz']
[182, 'Three Drives', 'Sylvie', 'Cruz']
[182, 'Sebastien Tellier', 'Sylvie', 'Cruz']
[182, 'Lonnie Gordon', 'Sylvie', 'Cruz']


## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [71]:
try:
    session.execute("""
   CREATE TABLE IF NOT EXISTS song_library_by_song(
        firstName text,
        lastName text,
        userId int,
        song text,
        PRIMARY KEY (song, userId)
    )
    """)
except Exception as e:
    print("Error creating table")
    print(e)              

In [77]:
file = 'event_datafile_new.csv'

# Insert expected data into song_plays_by_user_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library_by_song (firstName, lastName,userId ,song)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], int(line[10]), line[9]))
      

In [79]:
select("SELECT firstName, lastName FROM song_library_by_song WHERE song='All Hands Against His Own'")

In [89]:
# Using pandas to validate answer
import pandas as pd
event  = pd.read_csv('event_datafile_new.csv')
event[event['song'] == 'All Hands Against His Own']

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId


### Drop the tables before closing out the sessions

In [83]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS song_library")
session.execute("DROP TABLE IF EXISTS song_library_by_user")
session.execute("DROP TABLE IF EXISTS song_library_by_song")

### Close the session and cluster connection¶

In [84]:
session.shutdown()
cluster.shutdown()

In [90]:
!tar cvfz allfiles.tar.gz *

Project_1B_ Project_Template.ipynb
event_data/
event_data/2018-11-30-events.csv
event_data/2018-11-23-events.csv
event_data/2018-11-22-events.csv
event_data/2018-11-29-events.csv
event_data/2018-11-11-events.csv
event_data/2018-11-14-events.csv
event_data/2018-11-20-events.csv
event_data/2018-11-15-events.csv
event_data/2018-11-05-events.csv
event_data/2018-11-28-events.csv
event_data/2018-11-25-events.csv
event_data/2018-11-16-events.csv
event_data/2018-11-18-events.csv
event_data/.ipynb_checkpoints/
event_data/.ipynb_checkpoints/2018-11-30-events-checkpoint.csv
event_data/.ipynb_checkpoints/2018-11-20-events-checkpoint.csv
event_data/.ipynb_checkpoints/2018-11-27-events-checkpoint.csv
event_data/.ipynb_checkpoints/2018-11-02-events-checkpoint.csv
event_data/.ipynb_checkpoints/2018-11-01-events-checkpoint.csv
event_data/2018-11-24-events.csv
event_data/2018-11-04-events.csv
event_data/2018-11-19-events.csv
event_data/2018-11-26-events.csv
event_data/2018-11-12-events.csv
event_data/20